In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8bec240


In [2]:
n = 3

x = []
for i in range(n):
    x.append(ROOT.RooRealVar(f"x_{i}", f"x_{i}", 0., -5., 5.))

In [3]:
cov_mat = ROOT.TMatrixD(ROOT.TMatrixD.kUnit, ROOT.TMatrixD(n, n))
# print(cov_mat[0][0], cov_mat[0][1])
gen_model = ROOT.RooMultiVarGaussian("gauss", "gauss", ROOT.RooArgList(x), cov_mat)

In [4]:
p = []
y = []
squares = []
for i in range(n):
    p.append(ROOT.RooRealVar(f"p_{i}", f"p_{i}", 0., -1, 1))
    y.append(ROOT.RooAddition(f"y_{i}", f"y_{i}", x[i], p[i]))
    squares.append(ROOT.RooProduct(f"square_{i}", f"square_{i}", y[i], y[i]))

In [5]:
chi2 = ROOT.RooAddition("chi2_calc", "chi2_calc", ROOT.RooArgList(squares))

In [6]:
chi2_ndf = ROOT.RooConstVar("ndf", "ndf", n)
fit_model = ROOT.RooChiSquarePdf("chi2_pdf", "chi2_pdf", chi2, chi2_ndf)

In [7]:
# data = gen_model.generate(x, 1)
data = gen_model.generate(x, 100000)
data.Print()

RooDataSet::gaussData[x_0,x_1,x_2] = 100000 entries


In [8]:
# abuse ConditionalObservables to inhibit integration
nll = fit_model.createNLL(data, EvalBackend=("cpu"), ConditionalObservables=x)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(chi2_pdf) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 3.46611 ms


In [9]:
nll.Print("t")

0xb9428f0 RooAbsReal::RooEvaluatorWrapper = 3.55464e+07 [Auto,Dirty] 
  0xb96e690/-- RooAddition::nll_chi2_pdf_gaussData = 0 [Auto,Clean] 
    0xb8a4790/V- RooFit::Detail::RooNLLVarNew::RooNLLVarNew = 0 [Auto,Clean] 
      0xb9ef020/V- RooChiSquarePdf::chi2_pdf = 0 [ADIRTY] 
        0xb961d70/V- RooAddition::chi2_calc = 0 [ADIRTY] 
          0xb9a3e30/V- RooProduct::square_0 = 0 [ADIRTY] 
            0xb8f0a20/V- RooAddition::y_0 = 0 [ADIRTY] 
              0xb990f80/V- RooProduct::y_0_[x_0_x_p_0] = 0 [ADIRTY] 
                0xab751d0/V- RooRealVar::x_0 = 0
                0xada3130/V- RooRealVar::p_0 = 0
          0xb9a2040/V- RooProduct::square_1 = 0 [ADIRTY] 
            0xb963d80/V- RooAddition::y_1 = 0 [ADIRTY] 
              0x907b610/V- RooProduct::y_1_[x_1_x_p_1] = 0 [ADIRTY] 
                0xaad0490/V- RooRealVar::x_1 = 0
                0xad69200/V- RooRealVar::p_1 = 0
          0xb9622d0/V- RooProduct::square_2 = 0 [ADIRTY] 
            0xb989b80/V- RooAddition::y_2 = 0 

In [10]:
%time
ROOT.RooMinimizer(nll).migrad()

CPU times: user 8 μs, sys: 2 μs, total: 10 μs
Wall time: 19.3 μs


0

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_chi2_pdf_gaussData) Summation contains a RooNLLVar, using its error level
Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 160433.301683278492
Edm   = 0.000418166731033025002
Nfcn  = 130
p_0	  = -0.57944	 +/-  0.00317377	(limited)
p_1	  = -0.576352	 +/-  0.00315579	(limited)
p_2	  = -0.575662	 +/-  0.00315323	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 33.0982 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       35546372.14 Edm =                 0 NCalls =     13
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: NegativeG2LineSearch Done after 165.531 ms
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : 176129.6008
  Edm           : 24353.06221
  Internal parameters:	[    -0.2762070798    -0.9254464528    -0.2013579208]	
  Internal gradient  :	[      45168.26955      1801.825924      45813.96707]	
  Internal covariance matrix:
[[  1.7327176e-05              0              0]
 [              0  3.1845403e-05              0]
 [              0              0  2.9519202e-05]]]
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 176129.6008
  Ed

In [11]:
pll0 = nll.createProfile({x[0]})

In [12]:
frame = p[0].frame()
nll.plotOn(frame, ShiftToZero=True)
pll0.plotOn(frame, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[x_0]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_chi2_pdf_gaussData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[x_0]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[x_0]) minimum found at ()
........................................................................................................................................................................................................................................................

In [13]:
c = ROOT.TCanvas()
frame.Draw()
c.Draw()